In [1]:
import findspark
findspark.init()

In [2]:
import sys
import os

os.environ['HADOOP_HOME'] = "C:/Users/Husnain/Desktop/spark/spark-3.1.1-bin-hadoop2.7/spark-3.1.1-bin-hadoop2.7/hadoop"
    
sys.path.append("C:/Users/Husnain/Desktop/spark/spark-3.1.1-bin-hadoop2.7/spark-3.1.1-bin-hadoop2.7/hadoop/bin")

In [3]:
import pyspark
from pyspark.sql import SparkSession

In [4]:
# Import SparkSession from pyspark.sql
from pyspark.sql import SparkSession
from pyspark import SparkContext as sc

# Create my_spark
spark = SparkSession.builder.getOrCreate()

# Print my_spark
print(spark)


In [5]:
# Import the pyspark.sql.types library
from pyspark.sql.types import *

# Define a new schema using the StructType method
people_schema = StructType([
  # Define a StructField for each field
  StructField('name', StringType(), False),
  StructField('age', IntegerType(), False),
  StructField("city", StringType(),False)
])

In [6]:
# Load the CSV file
from pyspark.sql.functions import lower
aa_dfw_df = spark.read.format('csv').options(Header=True).load('cleaning_data/AA_DFW_2017_Departures_Short.csv.gz')
print(aa_dfw_df)
# Add the airport column using the F.lower() method
aa_dfw_df = aa_dfw_df.withColumn('airport', lower(aa_dfw_df['Destination Airport']))

# Drop the Destination Airport column
aa_dfw_df = aa_dfw_df.drop(aa_dfw_df['Destination Airport'])

# Show the DataFrame
aa_dfw_df.show()

DataFrame[Date (MM/DD/YYYY): string, Flight Number: string, Destination Airport: string, Actual elapsed time (Minutes): string]
+-----------------+-------------+-----------------------------+-------+
|Date (MM/DD/YYYY)|Flight Number|Actual elapsed time (Minutes)|airport|
+-----------------+-------------+-----------------------------+-------+
|       01/01/2017|         0005|                          537|    hnl|
|       01/01/2017|         0007|                          498|    ogg|
|       01/01/2017|         0037|                          241|    sfo|
|       01/01/2017|         0043|                          134|    dtw|
|       01/01/2017|         0051|                           88|    stl|
|       01/01/2017|         0060|                          149|    mia|
|       01/01/2017|         0071|                          203|    lax|
|       01/01/2017|         0074|                           76|    mem|
|       01/01/2017|         0081|                          123|    den|
|       

In [7]:
df1 = spark.read.csv("cleaning_data/AA_DFW_2015_Departures_Short/AA_DFW_2015_Departures_Short.csv")
df2 = spark.read.csv("cleaning_data/AA_DFW_2014_Departures_Short/AA_DFW_2014_Departures_Short.csv")

In [9]:
# View the row count of df1 and df2
print("df1 Count: %d" % df1.count())
print("df2 Count: %d" % df2.count())

# Combine the DataFrames into one
df3 = df1.union(df2)

# Save the df3 DataFrame in Parquet format
df3.write.parquet('AA_DFW_ALL.parquet', mode='overwrite')

# Read the Parquet file into a new DataFrame and run a count
print(spark.read.parquet('AA_DFW_ALL.parquet').count())

In [11]:
# Read the Parquet file into flights_df
flights_df = spark.read.parquet("AA_DFW_ALL.parquet")

# Register the temp table
flights_df.createOrReplaceTempView('flights')

# Run a SQL query of the average flight duration
avg_duration = spark.sql('SELECT avg(flight_duration) from flights').collect()[0]
print('The average flight time is: %d' % avg_duration)

In [12]:
from pyspark.sql.types import StructType, StructField, IntegerType,StringType

schema = StructType([
    StructField("DATE", StringType(), True),
    StructField("TITLE", StringType(), True),
    StructField("VOTER_NAME", StringType(), True)])

voter_df = spark.read.csv("cleaning_data/DallasCouncilVoters/DallasCouncilVoters.csv",header=False,schema=schema)

In [13]:
voter_df.show()

+----------+-------------+-------------------+
|      DATE|        TITLE|         VOTER_NAME|
+----------+-------------+-------------------+
|      DATE|        TITLE|         VOTER_NAME|
|02/08/2017|Councilmember|  Jennifer S. Gates|
|02/08/2017|Councilmember| Philip T. Kingston|
|02/08/2017|        Mayor|Michael S. Rawlings|
|02/08/2017|Councilmember|       Adam Medrano|
|02/08/2017|Councilmember|       Casey Thomas|
|02/08/2017|Councilmember|Carolyn King Arnold|
|02/08/2017|Councilmember|       Scott Griggs|
|02/08/2017|Councilmember|   B. Adam  McGough|
|02/08/2017|Councilmember|       Lee Kleinman|
|02/08/2017|Councilmember|      Sandy Greyson|
|02/08/2017|Councilmember|  Jennifer S. Gates|
|02/08/2017|Councilmember| Philip T. Kingston|
|02/08/2017|        Mayor|Michael S. Rawlings|
|02/08/2017|Councilmember|       Adam Medrano|
|02/08/2017|Councilmember|       Casey Thomas|
|02/08/2017|Councilmember|Carolyn King Arnold|
|02/08/2017|Councilmember| Rickey D. Callahan|
|01/11/2017|C

In [14]:
# Show the distinct VOTER_NAME entries
import pyspark.sql.functions as F
voter_df.select("VOTER_NAME").distinct().show(40, truncate=False)

# Filter voter_df where the VOTER_NAME is 1-20 characters in length
voter_df = voter_df.filter('length(VOTER_NAME) > 0 and length(VOTER_NAME) < 20')

# Filter out voter_df where the VOTER_NAME contains an underscore
voter_df = voter_df.filter(~ F.col('VOTER_NAME').contains("_"))

# Show the distinct VOTER_NAME entries again
voter_df.select("VOTER_NAME").distinct().show(40, truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|VOTER_NAME                                                                                                                                                                                                                                                                                                                                                                                                                 |
+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------

+-------------------+
|VOTER_NAME         |
+-------------------+
|Tennell Atkins     |
|Scott Griggs       |
|Scott  Griggs      |
|Sandy Greyson      |
|Michael S. Rawlings|
|Kevin Felder       |
|Adam Medrano       |
|Casey  Thomas      |
|Mark  Clayton      |
|Casey Thomas       |
|Sandy  Greyson     |
|Mark Clayton       |
|Jennifer S.  Gates |
|Tiffinni A. Young  |
|B. Adam  McGough   |
|Omar Narvaez       |
|Philip T. Kingston |
|Rickey D. Callahan |
|Dwaine R. Caraway  |
|Philip T.  Kingston|
|Jennifer S. Gates  |
|Lee M. Kleinman    |
|Monica R. Alonzo   |
|Rickey D.  Callahan|
|Carolyn King Arnold|
|Erik Wilson        |
|Lee Kleinman       |
+-------------------+



In [15]:
# Add a new column called splits separated on whitespace
voter_df = voter_df.withColumn("splits", F.split(voter_df.VOTER_NAME, '\s+'))

# Create a new column called first_name based on the first item in splits
voter_df = voter_df.withColumn("first_name", voter_df.splits.getItem(0))

# Get the last entry of the splits list and create a column called last_name
voter_df = voter_df.withColumn("last_name", voter_df.splits.getItem(F.size('splits')-1))

# Drop the splits column
voter_df = voter_df.drop('splits')

# Show the voter_df DataFrame
voter_df.show()

+----------+-------------+-------------------+----------+---------+
|      DATE|        TITLE|         VOTER_NAME|first_name|last_name|
+----------+-------------+-------------------+----------+---------+
|02/08/2017|Councilmember|  Jennifer S. Gates|  Jennifer|    Gates|
|02/08/2017|Councilmember| Philip T. Kingston|    Philip| Kingston|
|02/08/2017|        Mayor|Michael S. Rawlings|   Michael| Rawlings|
|02/08/2017|Councilmember|       Adam Medrano|      Adam|  Medrano|
|02/08/2017|Councilmember|       Casey Thomas|     Casey|   Thomas|
|02/08/2017|Councilmember|Carolyn King Arnold|   Carolyn|   Arnold|
|02/08/2017|Councilmember|       Scott Griggs|     Scott|   Griggs|
|02/08/2017|Councilmember|   B. Adam  McGough|        B.|  McGough|
|02/08/2017|Councilmember|       Lee Kleinman|       Lee| Kleinman|
|02/08/2017|Councilmember|      Sandy Greyson|     Sandy|  Greyson|
|02/08/2017|Councilmember|  Jennifer S. Gates|  Jennifer|    Gates|
|02/08/2017|Councilmember| Philip T. Kingston|  

In [16]:
# Add a column to voter_df for any voter with the title **Councilmember**
voter_df = voter_df.withColumn('random_val',
                               F.when(voter_df.TITLE == 'Councilmember', F.rand()))

# Show some of the DataFrame rows, noting whether the when clause worked
voter_df.show()

+----------+-------------+-------------------+----------+---------+--------------------+
|      DATE|        TITLE|         VOTER_NAME|first_name|last_name|          random_val|
+----------+-------------+-------------------+----------+---------+--------------------+
|02/08/2017|Councilmember|  Jennifer S. Gates|  Jennifer|    Gates|  0.4672698525957564|
|02/08/2017|Councilmember| Philip T. Kingston|    Philip| Kingston|   0.295785888425138|
|02/08/2017|        Mayor|Michael S. Rawlings|   Michael| Rawlings|                null|
|02/08/2017|Councilmember|       Adam Medrano|      Adam|  Medrano|  0.7829960775513511|
|02/08/2017|Councilmember|       Casey Thomas|     Casey|   Thomas|  0.4323406964593939|
|02/08/2017|Councilmember|Carolyn King Arnold|   Carolyn|   Arnold|  0.7157676135177211|
|02/08/2017|Councilmember|       Scott Griggs|     Scott|   Griggs|  0.3721183545139999|
|02/08/2017|Councilmember|   B. Adam  McGough|        B.|  McGough| 0.16041104040150478|
|02/08/2017|Councilme

In [17]:
# Add a column to voter_df for a voter based on their position
voter_df = voter_df.withColumn('random_val',
                               F.when(voter_df.TITLE == 'Councilmember', F.rand())
                               .when(voter_df.TITLE == "Mayor", 2)
                               .otherwise(0))

# Show some of the DataFrame rows
voter_df.show()

# Use the .filter() clause with random_val
voter_df.filter(voter_df.random_val==0).show()

+----------+-------------+-------------------+----------+---------+-------------------+
|      DATE|        TITLE|         VOTER_NAME|first_name|last_name|         random_val|
+----------+-------------+-------------------+----------+---------+-------------------+
|02/08/2017|Councilmember|  Jennifer S. Gates|  Jennifer|    Gates| 0.9731100292486756|
|02/08/2017|Councilmember| Philip T. Kingston|    Philip| Kingston|0.47025124772417337|
|02/08/2017|        Mayor|Michael S. Rawlings|   Michael| Rawlings|                2.0|
|02/08/2017|Councilmember|       Adam Medrano|      Adam|  Medrano|0.35539671731996436|
|02/08/2017|Councilmember|       Casey Thomas|     Casey|   Thomas| 0.8596267122194526|
|02/08/2017|Councilmember|Carolyn King Arnold|   Carolyn|   Arnold| 0.6279301050088159|
|02/08/2017|Councilmember|       Scott Griggs|     Scott|   Griggs|0.29305007704673325|
|02/08/2017|Councilmember|   B. Adam  McGough|        B.|  McGough| 0.9129196719554853|
|02/08/2017|Councilmember|      

In [18]:
voter_df = voter_df.withColumn("splits", F.split(voter_df.VOTER_NAME,'\s+'))

In [19]:
def getFirstAndMiddle(names):
  # Return a space separated string of names
  return ' '.join(names)

# Define the method as a UDF
udfFirstAndMiddle = F.udf(getFirstAndMiddle, StringType())

# Create a new column using your UDF
voter_df = voter_df.withColumn('first_and_middle_name', udfFirstAndMiddle(voter_df.splits))

# Show the DataFrame
voter_df.show()

+----------+-------------+-------------------+----------+---------+-------------------+--------------------+---------------------+
|      DATE|        TITLE|         VOTER_NAME|first_name|last_name|         random_val|              splits|first_and_middle_name|
+----------+-------------+-------------------+----------+---------+-------------------+--------------------+---------------------+
|02/08/2017|Councilmember|  Jennifer S. Gates|  Jennifer|    Gates| 0.9731100292486756|[Jennifer, S., Ga...|    Jennifer S. Gates|
|02/08/2017|Councilmember| Philip T. Kingston|    Philip| Kingston|0.47025124772417337|[Philip, T., King...|   Philip T. Kingston|
|02/08/2017|        Mayor|Michael S. Rawlings|   Michael| Rawlings|                2.0|[Michael, S., Raw...|  Michael S. Rawlings|
|02/08/2017|Councilmember|       Adam Medrano|      Adam|  Medrano|0.35539671731996436|     [Adam, Medrano]|         Adam Medrano|
|02/08/2017|Councilmember|       Casey Thomas|     Casey|   Thomas| 0.8596267122194

In [20]:
# Select all the unique council voters
voter_df = voter_df.select(voter_df["VOTER_NAME"]).distinct()

# Count the rows in voter_df
print("\nThere are %d rows in the voter_df DataFrame.\n" % voter_df.count())

# Add a ROW_ID
voter_df = voter_df.withColumn('ROW_ID', F.monotonically_increasing_id())

# Show the rows with 10 highest IDs in the set
voter_df.orderBy(voter_df.ROW_ID.desc()).show(10)


There are 27 rows in the voter_df DataFrame.

+-------------------+-------------+
|         VOTER_NAME|       ROW_ID|
+-------------------+-------------+
|       Lee Kleinman|1709396983808|
|        Erik Wilson|1700807049216|
|Carolyn King Arnold|1632087572480|
|Rickey D.  Callahan|1597727834112|
|   Monica R. Alonzo|1382979469312|
|    Lee M. Kleinman|1228360646656|
|  Jennifer S. Gates|1194000908288|
|Philip T.  Kingston|1185410973696|
|  Dwaine R. Caraway|1142461300736|
| Rickey D. Callahan|1125281431553|
+-------------------+-------------+
only showing top 10 rows



In [21]:
# Print the number of partitions in each DataFrame
print("\nThere are %d partitions in the voter_df DataFrame.\n" % voter_df.rdd.getNumPartitions())

# Add a ROW_ID field to each DataFrame
voter_df = voter_df.withColumn('ROW_ID', F.monotonically_increasing_id())

# Show the top 10 IDs in each DataFrame 
voter_df.orderBy(voter_df.ROW_ID.desc()).show(10)



There are 200 partitions in the voter_df DataFrame.

+-------------------+-------------+
|         VOTER_NAME|       ROW_ID|
+-------------------+-------------+
|       Lee Kleinman|1709396983808|
|        Erik Wilson|1700807049216|
|Carolyn King Arnold|1632087572480|
|Rickey D.  Callahan|1597727834112|
|   Monica R. Alonzo|1382979469312|
|    Lee M. Kleinman|1228360646656|
|  Jennifer S. Gates|1194000908288|
|Philip T.  Kingston|1185410973696|
|  Dwaine R. Caraway|1142461300736|
| Rickey D. Callahan|1125281431553|
+-------------------+-------------+
only showing top 10 rows



In [22]:
# Name of the Spark application instance
app_name = spark.conf.get('spark.app.name')

# Driver TCP port
driver_tcp_port = spark.conf.get('spark.driver.port')

# Number of join partitions
num_partitions = spark.conf.get('spark.sql.shuffle.partitions')

# Show the results
print("Name: %s" % app_name)
print("Driver TCP port: %s" % driver_tcp_port)
print("Number of partitions: %s" % num_partitions)

Name: pyspark-shell
Driver TCP port: 56051
Number of partitions: 200


In [23]:
# Store the number of partitions in variable
departures_df = spark.read.csv('cleaning_data/DallasCouncilVoters/DallasCouncilVoters.csv')
before = departures_df.rdd.getNumPartitions()
# Configure Spark to use 500 partitions
spark.conf.set('spark.sql.shuffle.partitions', 500)

# Recreate the DataFrame using the departures data file
departures_df = spark.read.csv('cleaning_data/DallasCouncilVoters/DallasCouncilVoters.csv').distinct()

# Print the number of partitions for each instance
print("Partition count before change: %d" % before)
print("Partition count after change: %d" % departures_df.rdd.getNumPartitions())

Partition count before change: 1
Partition count after change: 500


In [25]:
# Join the flights_df and aiports_df DataFrames
normal_df = departures_df.join(departures_df)

# Show the query plan
normal_df.explain()


== Physical Plan ==
BroadcastNestedLoopJoin BuildRight, Inner
:- *(2) HashAggregate(keys=[_c0#449, _c1#450, _c2#451], functions=[])
:  +- Exchange hashpartitioning(_c0#449, _c1#450, _c2#451, 500), ENSURE_REQUIREMENTS, [id=#520]
:     +- *(1) HashAggregate(keys=[_c0#449, _c1#450, _c2#451], functions=[])
:        +- FileScan csv [_c0#449,_c1#450,_c2#451] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[file:/C:/Users/Husnain/Desktop/DataEngineering/cleaning_data/DallasCouncilVoter..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<_c0:string,_c1:string,_c2:string>
+- BroadcastExchange IdentityBroadcastMode, [id=#551]
   +- *(4) HashAggregate(keys=[_c0#458, _c1#459, _c2#460], functions=[])
      +- ReusedExchange [_c0#458, _c1#459, _c2#460], Exchange hashpartitioning(_c0#449, _c1#450, _c2#451, 500), ENSURE_REQUIREMENTS, [id=#520]




In [26]:
# Import the broadcast method from pyspark.sql.functions
from pyspark.sql.functions import broadcast

# Join the flights_df and airports_df DataFrames using broadcasting
broadcast_df = departures_df.join(broadcast(departures_df) )

# Show the query plan and compare against the original
broadcast_df.explain()

== Physical Plan ==
BroadcastNestedLoopJoin BuildRight, Inner
:- *(2) HashAggregate(keys=[_c0#449, _c1#450, _c2#451], functions=[])
:  +- Exchange hashpartitioning(_c0#449, _c1#450, _c2#451, 500), ENSURE_REQUIREMENTS, [id=#593]
:     +- *(1) HashAggregate(keys=[_c0#449, _c1#450, _c2#451], functions=[])
:        +- FileScan csv [_c0#449,_c1#450,_c2#451] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[file:/C:/Users/Husnain/Desktop/DataEngineering/cleaning_data/DallasCouncilVoter..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<_c0:string,_c1:string,_c2:string>
+- BroadcastExchange IdentityBroadcastMode, [id=#624]
   +- *(4) HashAggregate(keys=[_c0#467, _c1#468, _c2#469], functions=[])
      +- ReusedExchange [_c0#467, _c1#468, _c2#469], Exchange hashpartitioning(_c0#449, _c1#450, _c2#451, 500), ENSURE_REQUIREMENTS, [id=#593]




In [27]:
path = 'cleaning_data/AA_DFW_2015_Departures_Short/AA_DFW_2015_Departures_Short.csv'

In [28]:
# Import the data to a DataFrame
departures_df = spark.read.csv(path, header=True)
# Remove any duration of 0
departures_df = departures_df.filter(departures_df[3]>0)

# Add an ID column
departures_df = departures_df.withColumn('id', F.monotonically_increasing_id())

# Write the file out to JSON format
# departures_df.write.json("output.json", mode='overwrite')


In [29]:
departures_df.show(5)

+-----------------+-------------+-------------------+-----------------------------+---+
|Date (MM/DD/YYYY)|Flight Number|Destination Airport|Actual elapsed time (Minutes)| id|
+-----------------+-------------+-------------------+-----------------------------+---+
|       01/01/2015|         0005|                HNL|                          526|  0|
|       01/01/2015|         0007|                OGG|                          517|  1|
|       01/01/2015|         0023|                SFO|                          233|  2|
|       01/01/2015|         0027|                LAS|                          165|  3|
|       01/01/2015|         0035|                HDN|                          178|  4|
+-----------------+-------------+-------------------+-----------------------------+---+
only showing top 5 rows

